### Faculdade de Engenharia Industrial - FEI

### Centro Universitário da Fundação Educacional Inaciana "Padre Sabóia de Medeiros" (FEI)


*FEI's Stricto Sensu Graduate Program in Electrical Engineering*

Concentration area: ARTIFICIAL INTELLIGENCE APPLIED TO AUTOMATION AND ROBOTICS

Master's thesis student Andre Luiz Florentino

***

## Select the processor type

In [1]:
# Select the way to load tensorflow in order to run on CPU only or CPU + GPU

import os

opc = 0
while str(opc) not in '12':
    print()
    print("1-) CPU only")
    print("2-) CPU + GPU")

    opc = input("\nSelect the processor: ")
    if opc.isdigit():
        opc = int(opc)
    else:
        opc = 0

if opc == 1:
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
    
    import tensorflow as tf

else:
    import tensorflow as tf
    print(tf.__version__)

    pd = tf.config.experimental.list_physical_devices()
    for i in pd:
        print(i)
    print('------------------------------------------------------------------------------------------')


    print(tf.config.list_physical_devices('GPU'))
    # [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

    print(tf.test.is_built_with_cuda)
    # <function is_built_with_cuda at 0x000001AA24AFEC10>

    print(tf.test.gpu_device_name())
    # /device:GPU:0

    #gvd = tf.config.get_visible_devices()
    for j in tf.config.get_visible_devices():
        print(j)
    # PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
    # PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')




1-) CPU only
2-) CPU + GPU

Select the processor: 1


## Check for GPU

In [2]:
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

No GPU found


# Chapter 8: ESR data preparation

***

## Import modules

In [3]:
import psutil
import csv

import pandas          as pd
import numpy           as np

from collections     import Counter
%load_ext memory_profiler

pd.set_option('display.max_columns', 9)
pd.set_option('display.width', 300)
pd.set_option('display.max_colwidth', 120)

#%load_ext scalene

## Data preparation

In [4]:
path        = "C:\\Andre_Florentino\\03_particular\\04_mestrado-FEI\\98_dataset\\US8K_AV\\"
path_models = "C:\\Andre_Florentino\\03_particular\\04_mestrado-FEI\\96_code_cv\\ESR\\Saved_models_fold_1_validation\\"
path_arrays = "C:\\Andre_Florentino\\03_particular\\04_mestrado-FEI\\96_code_cv\\ESR\\Arrays\\"

nom_dataset               = 'US8K_AV' 
csv_file                  = 'US8K_AV.csv'
fold_name                 = '01'

pkl_features              = 'US8K_AV_features_original.pkl'
pkl_aug_wind_features     = 'US8K_AV_features_windowed.pkl' # not augmented, just windowed
pkl_aug_features          = 'US8K_AV_features_augmented_no_windowing.pkl'

pkl_aug_wind_features_CNN = 'US8K_AV_features_aug_wind_CNN_2D.pkl'

DB_ori_from_pkl           = pd.read_pickle(path + pkl_features)

In [5]:
print(type(DB_ori_from_pkl['Fold'][0][0]))
print(type(DB_ori_from_pkl['Class_OHEV'][0][0]))
print(type(DB_ori_from_pkl['Class_OHEV'][0]))

<class 'str'>
<class 'int'>
<class 'list'>


In [6]:
DB_ori_from_pkl

,Audio,Class_categorical,Class_OHEV,Fold,...,TONNETZ_std_6,TONNETZ_median_6,TONNETZ_skew_6,TONNETZ_kurtosis_6
0,"[-0.0034710653, -0.0050192624, -0.004654482, -0.0049833283, -0.0038681468, -0.0023575649, -0.00025486574, 0.00135406...",dog_bark,"[0, 0, 0, 1, 0]",05,...,0.024244,-0.049756,0.759231,-0.024962
1,"[0.0027405696, 0.0036386526, 0.0023064336, 0.0023546093, 0.0025339962, 0.0030277965, 0.0027998006, 0.0028241847, 0.0...",children_playing,"[0, 0, 1, 0, 0]",05,...,0.018670,-0.018241,0.275013,-0.497943
2,"[-0.0027212622, -0.0030217567, -0.0016039186, -0.0012952066, -0.001044448, -0.0011087725, -0.0006573542, -0.00120615...",children_playing,"[0, 0, 1, 0, 0]",05,...,0.018733,0.009139,-0.205466,-0.212879
3,"[0.00015050662, -0.00023231667, -0.0013628466, -0.002236663, -0.0027237535, -0.002267104, -0.0012131947, -0.00148792...",children_playing,"[0, 0, 1, 0, 0]",05,...,0.027117,0.007208,-0.769827,0.623153
4,"[0.0012501464, 0.00428795, 0.0062407875, 0.0055125337, 0.003776363, 0.0018791268, 0.0014532767, 0.0026125484, 0.0038...",children_playing,"[0, 0, 1, 0, 0]",05,...,0.025776,0.050245,0.208739,-1.062755
...,...,...,...,...,...,...,...,...,...
4353,"[-0.00048753922, -0.0029013595, -0.0060868477, -0.0077148406, -0.008069417, -0.008534078, -0.008617738, -0.007554673...",car_horn,"[0, 1, 0, 0, 0]",07,...,0.022642,0.008688,0.097807,-0.866208
4354,"[0.0018769287, 0.0028417986, 0.0028369268, 0.003081231, 0.0018484257, 0.0010519021, 0.00044334529, 0.0001646977, -3....",car_horn,"[0, 1, 0, 0, 0]",07,...,0.029405,0.020661,-0.220407,1.997790
4355,"[-0.013202029, -0.020061549, -0.016201336, -0.014081302, -0.0110550765, -0.010160886, -0.012472121, -0.013130652, -0...",car_horn,"[0, 1, 0, 0, 0]",07,...,0.034195,-0.002075,0.725307,-0.462707
4356,"[0.0008518654, -0.003709917, -0.009393071, -0.009993944, -0.0063429633, -0.0011706308, 0.0025487966, 0.0038857076, 0...",car_horn,"[0, 1, 0, 0, 0]",07,...,0.029228,-0.005832,-0.330073,-0.796058


In [7]:
DB_ori_from_pkl = DB_ori_from_pkl[['Audio', 'Class_categorical', 'Class_OHEV', 'Fold']]
DB_ori_from_pkl

,Audio,Class_categorical,Class_OHEV,Fold
0,"[-0.0034710653, -0.0050192624, -0.004654482, -0.0049833283, -0.0038681468, -0.0023575649, -0.00025486574, 0.00135406...",dog_bark,"[0, 0, 0, 1, 0]",05
1,"[0.0027405696, 0.0036386526, 0.0023064336, 0.0023546093, 0.0025339962, 0.0030277965, 0.0027998006, 0.0028241847, 0.0...",children_playing,"[0, 0, 1, 0, 0]",05
2,"[-0.0027212622, -0.0030217567, -0.0016039186, -0.0012952066, -0.001044448, -0.0011087725, -0.0006573542, -0.00120615...",children_playing,"[0, 0, 1, 0, 0]",05
3,"[0.00015050662, -0.00023231667, -0.0013628466, -0.002236663, -0.0027237535, -0.002267104, -0.0012131947, -0.00148792...",children_playing,"[0, 0, 1, 0, 0]",05
4,"[0.0012501464, 0.00428795, 0.0062407875, 0.0055125337, 0.003776363, 0.0018791268, 0.0014532767, 0.0026125484, 0.0038...",children_playing,"[0, 0, 1, 0, 0]",05
...,...,...,...,...
4353,"[-0.00048753922, -0.0029013595, -0.0060868477, -0.0077148406, -0.008069417, -0.008534078, -0.008617738, -0.007554673...",car_horn,"[0, 1, 0, 0, 0]",07
4354,"[0.0018769287, 0.0028417986, 0.0028369268, 0.003081231, 0.0018484257, 0.0010519021, 0.00044334529, 0.0001646977, -3....",car_horn,"[0, 1, 0, 0, 0]",07
4355,"[-0.013202029, -0.020061549, -0.016201336, -0.014081302, -0.0110550765, -0.010160886, -0.012472121, -0.013130652, -0...",car_horn,"[0, 1, 0, 0, 0]",07
4356,"[0.0008518654, -0.003709917, -0.009393071, -0.009993944, -0.0063429633, -0.0011706308, 0.0025487966, 0.0038857076, 0...",car_horn,"[0, 1, 0, 0, 0]",07


In [8]:
for fold in np.unique(DB_ori_from_pkl['Fold']):
    print(f"Validation fold: {fold}")
    
    valsize = len(DB_ori_from_pkl[DB_ori_from_pkl['Fold'] == fold])
    trnsize = len(DB_ori_from_pkl[DB_ori_from_pkl['Fold'] != fold])
    print(f'dbComplete_VAL size: {valsize}')
    print(f'dbComplete size: {trnsize}')
    print()

Validation fold: 01
dbComplete_VAL size: 424
dbComplete size: 3934

Validation fold: 02
dbComplete_VAL size: 432
dbComplete size: 3926

Validation fold: 03
dbComplete_VAL size: 465
dbComplete size: 3893

Validation fold: 04
dbComplete_VAL size: 538
dbComplete size: 3820

Validation fold: 05
dbComplete_VAL size: 476
dbComplete size: 3882

Validation fold: 06
dbComplete_VAL size: 388
dbComplete size: 3970

Validation fold: 07
dbComplete_VAL size: 402
dbComplete size: 3956

Validation fold: 08
dbComplete_VAL size: 405
dbComplete size: 3953

Validation fold: 09
dbComplete_VAL size: 411
dbComplete size: 3947

Validation fold: 10
dbComplete_VAL size: 417
dbComplete size: 3941



In [9]:
# Group by the class and get one random sample of each class
k = DB_ori_from_pkl.groupby('Class_categorical')['Class_OHEV'].apply(lambda s: s.sample(1))
print(k)

# Convert the pandas series into a dataframe
temp_k_df = k.reset_index()

# Delete the index from the grouppby result
del temp_k_df['level_1']

# Set the "Class" as the dataframe index
temp_k_df.set_index("Class_categorical", inplace=True)

# Convert the dataframe to a dictionary (Class: Class_encoder)
encoder_dict = temp_k_df["Class_OHEV"].to_dict()
encoder_dict

Class_categorical      
background         105     [1, 0, 0, 0, 0]
car_horn           1428    [0, 1, 0, 0, 0]
children_playing   2268    [0, 0, 1, 0, 0]
dog_bark           2929    [0, 0, 0, 1, 0]
siren              1370    [0, 0, 0, 0, 1]
Name: Class_OHEV, dtype: object


{'background': [1, 0, 0, 0, 0],
 'car_horn': [0, 1, 0, 0, 0],
 'children_playing': [0, 0, 1, 0, 0],
 'dog_bark': [0, 0, 0, 1, 0],
 'siren': [0, 0, 0, 0, 1]}

In [10]:
# Number of classes in the dataset

num_classes = len(encoder_dict.keys())
num_classes

5

In [11]:
for col in DB_ori_from_pkl.columns:
    print(col)

Audio
Class_categorical
Class_OHEV
Fold


In [12]:
# Separate 1 fold for validation and create a DB for the training / testing

DB_ori_from_pkl_VAL = DB_ori_from_pkl[DB_ori_from_pkl['Fold'] == fold_name].copy()
DB_ori_from_pkl_TRN = DB_ori_from_pkl[DB_ori_from_pkl['Fold'] != fold_name].copy()

In [13]:
print(len(DB_ori_from_pkl_VAL))
print(len(DB_ori_from_pkl_TRN))
print('Total: ', len(DB_ori_from_pkl_VAL) + len(DB_ori_from_pkl_TRN),'\n')

424
3934
Total:  4358 



In [14]:
audio_val = np.array(DB_ori_from_pkl_VAL.Audio.to_list())
y_val     = np.array(DB_ori_from_pkl_VAL.Class_OHEV.to_list())
y_val_cat = np.array(DB_ori_from_pkl_VAL.Class_categorical.to_list())

In [15]:
print(audio_val.shape)
print(y_val.shape)
print(y_val_cat.shape)

(424, 88200)
(424, 5)
(424,)


In [16]:
Counter_val_cat  = Counter(y_val_cat)
Counter_val      = Counter(map(tuple, y_val))

In [17]:
print('Validation categorical samples')
Counter_val_cat

Validation categorical samples


Counter({'dog_bark': 100,
         'background': 102,
         'children_playing': 100,
         'siren': 86,
         'car_horn': 36})

In [18]:
print('Validation OHEV samples')
Counter_val

Validation OHEV samples


Counter({(0, 0, 0, 1, 0): 100,
         (1, 0, 0, 0, 0): 102,
         (0, 0, 1, 0, 0): 100,
         (0, 0, 0, 0, 1): 86,
         (0, 1, 0, 0, 0): 36})

### Results from the file 08_ESR_data_preparation.ipynb for confirming the predictions

In [19]:
predict_audioClip_dicR = np.load(path_arrays + '_predict_audioClip_dic.npy', allow_pickle='TRUE').item()
predict_audio_dicR     = np.load(path_arrays + '_predict_audio_dic.npy',     allow_pickle='TRUE').item()

In [20]:
saved_predict_val_LR    = np.genfromtxt(path_arrays + '_saved_predict_val_LR.csv',    delimiter=',', dtype = str)
saved_predict_val_LR    = np.char.strip(saved_predict_val_LR, "'")
saved_predict_val_RF    = np.genfromtxt(path_arrays + '_saved_predict_val_RF.csv',    delimiter=',', dtype = str)
saved_predict_val_RF    = np.char.strip(saved_predict_val_RF, "'")
saved_predict_val_ANN   = np.genfromtxt(path_arrays + '_saved_predict_val_ANN.csv',   delimiter=',', dtype = int)
saved_predict_val_CNN1D = np.genfromtxt(path_arrays + '_saved_predict_val_CNN1D.csv', delimiter=',', dtype = int)
saved_predict_val_CNN2D = np.genfromtxt(path_arrays + '_saved_predict_val_CNN2D.csv', delimiter=',', dtype = int)

In [21]:
print(np.array_equal(saved_predict_val_LR,    predict_audioClip_dicR['predict_audioClip_LR']))
print(np.array_equal(saved_predict_val_RF,    predict_audioClip_dicR['predict_audioClip_RF']))
print(np.array_equal(saved_predict_val_ANN,   predict_audioClip_dicR['predict_audioClip_ANN']))
print(np.array_equal(saved_predict_val_CNN1D, predict_audioClip_dicR['predict_audioClip_CNN1D']))
print(np.array_equal(saved_predict_val_CNN2D, predict_audioClip_dicR['predict_audioClip_CNN2D']))

True
True
True
True
True


In [22]:
# Retrive the ground truth label of each original audio

y_val_audio_cat = np.genfromtxt(path_arrays + '_y_val_audio_cat.csv', delimiter=',', dtype = str)
y_val_audio_cat = np.char.strip(y_val_audio_cat, "'")
y_val_audio_enc = np.genfromtxt(path_arrays + '_y_val_audio_enc.csv', delimiter=',', dtype = int)

In [23]:
# Retrive the ground truth label of each windowed audio

y_val_audio_wind_cat = np.genfromtxt(path_arrays + '_y_val_audio_wind_cat.csv', delimiter=',', dtype = str)
y_val_audio_wind_cat = np.char.strip(y_val_audio_wind_cat, "'")
y_val_audio_wind_enc = np.genfromtxt(path_arrays + '_y_val_audio_wind_enc.csv', delimiter=',', dtype = int)

### Evaluation algorithm

https://www.geeksforgeeks.org/magic-commands-for-profiling-in-jupyter-notebook/

In [27]:
from MT_ESR_evaluation import ESR_evaluation

In [28]:
Classifiers = ['LR', 'RF', 'ANN', 'CNN1D', 'CNN2D'] 

In [29]:
# Memory profiler

def func():
    ESR_EVAL = ESR_evaluation(audio_val[0:10], 'CNN2D')
    predictions    = np.array(ESR_EVAL.predictions)
    totalPredTime  = np.array(ESR_EVAL.totalPredTime)
    
    return predictions, totalPredTime

In [40]:
%memit func()

Sample...: 1
dog_bark
Total predict time:..: 106.7891ms

Sample...: 2
dog_bark
Total predict time:..: 36.5067ms

Sample...: 3
dog_bark
Total predict time:..: 30.0016ms

Sample...: 4
dog_bark
Total predict time:..: 34.7186ms

Sample...: 5
dog_bark
Total predict time:..: 29.4928ms

Sample...: 6
dog_bark
Total predict time:..: 33.1500ms

Sample...: 7
dog_bark
Total predict time:..: 27.7034ms

Sample...: 8
dog_bark
Total predict time:..: 26.7538ms

Sample...: 9
dog_bark
Total predict time:..: 31.9534ms

Sample...: 10
dog_bark
Total predict time:..: 27.2383ms

Sample...: 11
dog_bark
Total predict time:..: 26.7410ms

Sample...: 12
dog_bark
Total predict time:..: 26.5811ms

Sample...: 13
dog_bark
Total predict time:..: 34.7451ms

Sample...: 14
dog_bark
Total predict time:..: 26.7646ms

Sample...: 15
dog_bark
Total predict time:..: 31.7047ms

Sample...: 16
dog_bark
Total predict time:..: 26.7548ms

Sample...: 17
dog_bark
Total predict time:..: 27.6995ms

Sample...: 18
dog_bark
Total predict ti

In [46]:
ESR_EVAL = ESR_evaluation(audio_val, 'CNN2D')
predictions    = np.array(ESR_EVAL.predictions)
totalPredTime  = np.array(ESR_EVAL.totalPredTime)

Sample...: 1
dog_bark
Total predict time:..: 100.8746ms

Sample...: 2
dog_bark
Total predict time:..: 35.9662ms

Sample...: 3
dog_bark
Total predict time:..: 31.8016ms

Sample...: 4
dog_bark
Total predict time:..: 29.0640ms

Sample...: 5
dog_bark
Total predict time:..: 31.1396ms

Sample...: 6
dog_bark
Total predict time:..: 36.8570ms

Sample...: 7
dog_bark
Total predict time:..: 32.0452ms

Sample...: 8
dog_bark
Total predict time:..: 35.8212ms

Sample...: 9
dog_bark
Total predict time:..: 30.7779ms

Sample...: 10
dog_bark
Total predict time:..: 32.9470ms

Sample...: 11
dog_bark
Total predict time:..: 32.4750ms

Sample...: 12
dog_bark
Total predict time:..: 40.7895ms

Sample...: 13
dog_bark
Total predict time:..: 31.8268ms

Sample...: 14
dog_bark
Total predict time:..: 28.2969ms

Sample...: 15
dog_bark
Total predict time:..: 28.5093ms

Sample...: 16
dog_bark
Total predict time:..: 27.6800ms

Sample...: 17
dog_bark
Total predict time:..: 31.8644ms

Sample...: 18
dog_bark
Total predict ti

background
Total predict time:..: 33.9396ms

Sample...: 142
background
Total predict time:..: 31.8513ms

Sample...: 143
children_playing
Total predict time:..: 29.6311ms

Sample...: 144
background
Total predict time:..: 36.3513ms

Sample...: 145
background
Total predict time:..: 26.5927ms

Sample...: 146
background
Total predict time:..: 27.9321ms

Sample...: 147
dog_bark
Total predict time:..: 27.6982ms

Sample...: 148
children_playing
Total predict time:..: 35.0022ms

Sample...: 149
children_playing
Total predict time:..: 29.5868ms

Sample...: 150
children_playing
Total predict time:..: 29.5154ms

Sample...: 151
children_playing
Total predict time:..: 40.8688ms

Sample...: 152
children_playing
Total predict time:..: 35.9493ms

Sample...: 153
children_playing
Total predict time:..: 28.2892ms

Sample...: 154
children_playing
Total predict time:..: 29.2980ms

Sample...: 155
children_playing
Total predict time:..: 31.8552ms

Sample...: 156
children_playing
Total predict time:..: 34.0986m

children_playing
Total predict time:..: 34.4925ms

Sample...: 276
children_playing
Total predict time:..: 28.5953ms

Sample...: 277
children_playing
Total predict time:..: 32.8472ms

Sample...: 278
children_playing
Total predict time:..: 31.5465ms

Sample...: 279
children_playing
Total predict time:..: 28.6520ms

Sample...: 280
children_playing
Total predict time:..: 27.1562ms

Sample...: 281
children_playing
Total predict time:..: 26.2917ms

Sample...: 282
children_playing
Total predict time:..: 33.1007ms

Sample...: 283
children_playing
Total predict time:..: 33.7970ms

Sample...: 284
children_playing
Total predict time:..: 30.5672ms

Sample...: 285
children_playing
Total predict time:..: 33.2894ms

Sample...: 286
children_playing
Total predict time:..: 32.2765ms

Sample...: 287
children_playing
Total predict time:..: 28.8260ms

Sample...: 288
background
Total predict time:..: 30.1210ms

Sample...: 289
background
Total predict time:..: 30.3386ms

Sample...: 290
background
Total predi

children_playing
Total predict time:..: 33.5041ms

Sample...: 420
children_playing
Total predict time:..: 27.2674ms

Sample...: 421
siren
Total predict time:..: 35.9340ms

Sample...: 422
children_playing
Total predict time:..: 29.0561ms

Sample...: 423
children_playing
Total predict time:..: 30.7533ms

Sample...: 424
children_playing
Total predict time:..: 31.7379ms

Sample...: 425
siren
Total predict time:..: 29.1425ms

Sample...: 426
siren
Total predict time:..: 33.1404ms

Sample...: 427
siren
Total predict time:..: 34.3646ms

Sample...: 428
siren
Total predict time:..: 30.4815ms

Sample...: 429
siren
Total predict time:..: 35.0900ms

Sample...: 430
siren
Total predict time:..: 34.7495ms

Sample...: 431
siren
Total predict time:..: 33.0101ms

Sample...: 432
siren
Total predict time:..: 36.8789ms

Sample...: 433
siren
Total predict time:..: 31.4188ms

Sample...: 434
siren
Total predict time:..: 27.5376ms

Sample...: 435
siren
Total predict time:..: 29.6939ms

Sample...: 436
siren
Tota

children_playing
Total predict time:..: 36.4323ms

Sample...: 562
children_playing
Total predict time:..: 31.3082ms

Sample...: 563
children_playing
Total predict time:..: 31.6742ms

Sample...: 564
children_playing
Total predict time:..: 30.4387ms

Sample...: 565
children_playing
Total predict time:..: 37.2625ms

Sample...: 566
children_playing
Total predict time:..: 33.2173ms

Sample...: 567
children_playing
Total predict time:..: 31.3898ms

Sample...: 568
children_playing
Total predict time:..: 31.4678ms

Sample...: 569
children_playing
Total predict time:..: 32.9284ms

Sample...: 570
children_playing
Total predict time:..: 29.4118ms

Sample...: 571
children_playing
Total predict time:..: 28.4793ms

Sample...: 572
children_playing
Total predict time:..: 30.0544ms

Sample...: 573
children_playing
Total predict time:..: 29.2655ms

Sample...: 574
children_playing
Total predict time:..: 31.5199ms

Sample...: 575
children_playing
Total predict time:..: 31.0563ms

Sample...: 576
children_p

...

children_playing
Total predict time:..: 38.4217ms

Sample...: 2363
siren
Total predict time:..: 32.6103ms

Sample...: 2364
siren
Total predict time:..: 31.9329ms

Sample...: 2365
children_playing
Total predict time:..: 28.3682ms

Sample...: 2366
children_playing
Total predict time:..: 32.3520ms

Sample...: 2367
siren
Total predict time:..: 27.4797ms

Sample...: 2368
children_playing
Total predict time:..: 27.0726ms

Sample...: 2369
children_playing
Total predict time:..: 29.2474ms

Sample...: 2370
children_playing
Total predict time:..: 38.9526ms

Sample...: 2371
children_playing
Total predict time:..: 35.5861ms

Sample...: 2372
children_playing
Total predict time:..: 31.7158ms

Sample...: 2373
children_playing
Total predict time:..: 26.9965ms

Sample...: 2374
children_playing
Total predict time:..: 31.3393ms

Sample...: 2375
children_playing
Total predict time:..: 27.3483ms

Sample...: 2376
children_playing
Total predict time:..: 28.8112ms

Sample...: 2377
children_playing
Total predi

dog_bark
Total predict time:..: 27.8571ms

Sample...: 2501
dog_bark
Total predict time:..: 31.0647ms

Sample...: 2502
dog_bark
Total predict time:..: 30.8386ms

Sample...: 2503
dog_bark
Total predict time:..: 32.5324ms

Sample...: 2504
dog_bark
Total predict time:..: 28.6678ms

Sample...: 2505
dog_bark
Total predict time:..: 26.3170ms

Sample...: 2506
dog_bark
Total predict time:..: 32.0179ms

Sample...: 2507
children_playing
Total predict time:..: 30.3454ms

Sample...: 2508
dog_bark
Total predict time:..: 31.5688ms

Sample...: 2509
dog_bark
Total predict time:..: 37.3577ms

Sample...: 2510
dog_bark
Total predict time:..: 27.6483ms

Sample...: 2511
dog_bark
Total predict time:..: 34.1319ms

Sample...: 2512
dog_bark
Total predict time:..: 31.4239ms

Sample...: 2513
dog_bark
Total predict time:..: 40.6502ms

Sample...: 2514
dog_bark
Total predict time:..: 34.2587ms

Sample...: 2515
dog_bark
Total predict time:..: 36.4671ms

Sample...: 2516
dog_bark
Total predict time:..: 29.5283ms

Sampl

dog_bark
Total predict time:..: 31.9729ms

Sample...: 2641
dog_bark
Total predict time:..: 36.5081ms

Sample...: 2642
dog_bark
Total predict time:..: 27.5678ms

Sample...: 2643
dog_bark
Total predict time:..: 33.1108ms

Sample...: 2644
dog_bark
Total predict time:..: 35.2779ms

Sample...: 2645
dog_bark
Total predict time:..: 31.5642ms

Sample...: 2646
dog_bark
Total predict time:..: 29.4491ms

Sample...: 2647
dog_bark
Total predict time:..: 34.5233ms

Sample...: 2648
dog_bark
Total predict time:..: 37.2702ms

Sample...: 2649
dog_bark
Total predict time:..: 32.4066ms

Sample...: 2650
dog_bark
Total predict time:..: 41.1371ms

Sample...: 2651
dog_bark
Total predict time:..: 27.9039ms

Sample...: 2652
dog_bark
Total predict time:..: 33.0022ms

Sample...: 2653
dog_bark
Total predict time:..: 36.4456ms

Sample...: 2654
dog_bark
Total predict time:..: 34.8437ms

Sample...: 2655
dog_bark
Total predict time:..: 32.9661ms

Sample...: 2656
dog_bark
Total predict time:..: 36.4653ms

Sample...: 26

background
Total predict time:..: 30.6523ms

Sample...: 2782
background
Total predict time:..: 28.3965ms

Sample...: 2783
background
Total predict time:..: 27.2493ms

Sample...: 2784
background
Total predict time:..: 27.9575ms

Sample...: 2785
background
Total predict time:..: 36.4672ms

Sample...: 2786
background
Total predict time:..: 31.0608ms

Sample...: 2787
background
Total predict time:..: 32.1145ms

Sample...: 2788
background
Total predict time:..: 27.0448ms

Sample...: 2789
background
Total predict time:..: 27.4279ms

Sample...: 2790
background
Total predict time:..: 32.0844ms

Sample...: 2791
background
Total predict time:..: 31.1955ms

Sample...: 2792
background
Total predict time:..: 29.7855ms

Sample...: 2793
background
Total predict time:..: 26.3058ms

Sample...: 2794
background
Total predict time:..: 30.4672ms

Sample...: 2795
background
Total predict time:..: 30.1088ms

Sample...: 2796
background
Total predict time:..: 31.0806ms

Sample...: 2797
background
Total predict

children_playing
Total predict time:..: 31.8544ms

Sample...: 2920
background
Total predict time:..: 31.4068ms

Sample...: 2921
background
Total predict time:..: 165.4065ms

Sample...: 2922
background
Total predict time:..: 29.7480ms

Sample...: 2923
background
Total predict time:..: 32.6335ms

Sample...: 2924
background
Total predict time:..: 31.2312ms

Sample...: 2925
background
Total predict time:..: 32.2256ms

Sample...: 2926
background
Total predict time:..: 29.5600ms

Sample...: 2927
background
Total predict time:..: 27.7956ms

Sample...: 2928
background
Total predict time:..: 27.6423ms

Sample...: 2929
background
Total predict time:..: 29.7649ms

Sample...: 2930
background
Total predict time:..: 35.8466ms

Sample...: 2931
background
Total predict time:..: 29.6939ms

Sample...: 2932
background
Total predict time:..: 28.3362ms

Sample...: 2933
background
Total predict time:..: 27.3739ms

Sample...: 2934
background
Total predict time:..: 33.4556ms

Sample...: 2935
background
Total 

In [47]:
predictions

array([3, 3, 3, ..., 0, 0, 0], dtype=int64)

In [48]:
predictions.shape

(2968,)

In [49]:
totalPredTime

array([100.8746,  35.9662,  31.8016, ...,  34.3822,  30.2117,  27.1859])

In [50]:
np.average(totalPredTime)

31.927048955525606

In [51]:
# Check if the algorithm is outputing the same results

print(np.array_equal(saved_predict_val_CNN2D, predictions))

True


In [ ]:
dic_evalT = {}

for i in range(10):
    print("=================================================================")
    print(f'Batch...: {i}\n')
    dic_eval  = {}
    for classifier in Classifiers:
        print(f'Classifier: {classifier}\n')
        ESR_EVAL       = ESR_evaluation(audio_val, classifier)
        predictions    = np.array(ESR_EVAL.predictions)
        totalPredTime  = np.array(ESR_EVAL.totalPredTime)

        dic_eval[f'predictions_{classifier}']   = predictions
        dic_eval[f'totalPredTime_{classifier}'] = totalPredTime
        
    dic_evalT[f'predictions_{i}'] = dic_eval

In [ ]:
dic_evalT

In [ ]:
np.save(path_arrays + '_dic_evalT.npy', dic_evalT)

In [ ]:
dic_evalS = np.load(path_arrays + '_dic_evalT.npy', allow_pickle='TRUE').item()
dic_evalS

In [ ]:
dic_evalS.keys()

In [ ]:
for key in dic_evalS.keys():
    print(key)
    print(dic_evalS[key]['totalPredTime_CNN1D'])

In [ ]:
# Write the total prediction time of each classifier in each loop

for key in dic_evalS.keys():

    with open(path_arrays + '_totalPredTime.csv', mode='w', newline='') as file:
        writer = csv.writer(file)

        # Write header LOGISTIC REGRESSION
        writer.writerow(['Key', 'Total Prediction Time', 'totalPredTime_LR'])
        # Write data
        for key in dic_evalS.keys():
            total_pred_time = np.array(dic_evalS[key]['totalPredTime_LR']).tolist()
            writer.writerow([key, total_pred_time])
            
        # Write header RANDOM FOREST
        writer.writerow(['Key', 'Total Prediction Time', 'totalPredTime_RF'])
        # Write data
        for key in dic_evalS.keys():
            total_pred_time = np.array(dic_evalS[key]['totalPredTime_RF']).tolist()
            writer.writerow([key, total_pred_time])
            
        # Write header LOGISTIC ARTIFICIAL NEURAL NETWORK
        writer.writerow(['Key', 'Total Prediction Time', 'totalPredTime_ANN'])
        # Write data
        for key in dic_evalS.keys():
            total_pred_time = np.array(dic_evalS[key]['totalPredTime_ANN']).tolist()
            writer.writerow([key, total_pred_time])
            
        # Write header CONVOLUTIONAL NEURAL NETWORK 1D
        writer.writerow(['Key', 'Total Prediction Time', 'totalPredTime_CNN1D'])
        # Write data
        for key in dic_evalS.keys():
            total_pred_time = np.array(dic_evalS[key]['totalPredTime_CNN1D']).tolist()
            writer.writerow([key, total_pred_time])
            
        # Write header CONVOLUTIONAL NEURAL NETWORK 2D
        writer.writerow(['Key', 'Total Prediction Time', 'totalPredTime_CNN2D'])
        # Write data
        for key in dic_evalS.keys():
            total_pred_time = np.array(dic_evalS[key]['totalPredTime_CNN2D']).tolist()
            writer.writerow([key, total_pred_time])

print('Output has been written to _totalPredTime.csv')

In [ ]:
predictionsName  = ['predictions_LR', 'predictions_RF', 'predictions_ANN', 'predictions_CNN1D', 'predictions_CNN2D']

In [ ]:
# Check predictions

for key in dic_evalS.keys():
    savedPredictions = ['predict_audioClip_LR', 'predict_audioClip_RF', 'predict_audioClip_ANN', 'predict_audioClip_CNN1D', 'predict_audioClip_CNN2D']
    print(f'\n{key}\n')
    for prediction, savedPrediction in zip(predictionsName,savedPredictions):
        print(prediction)
        checkPred = np.array(dic_evalS[key][prediction])
        savedPred = predict_audioClip_dicR[savedPrediction]
        print(np.array_equal(checkPred, savedPred))


In [ ]:
#process = psutil.Process(os.getpid())

#rssI    = (process.memory_full_info().uss/1024)/1024

#func()

#rssF    = (process.memory_full_info().uss/1024)/1024 
#rssT    = "{:.2f}".format(rssF - rssI)

#print('Memória RSS utilizada: ', rssT, 'MB')

In [ ]:
# inner psutil function
#def process_memory():
#    process  = psutil.Process(os.getpid())
#  mem_info = process.memory_full_info()
#    return (mem_info.uss/1024)/1024
 
# decorator function
#def profile(func):
#    def wrapper(*args, **kwargs):
# 
#        mem_before = process_memory()
#        result = func(*args, **kwargs)
#        mem_after  = process_memory()
#        print("{}:consumed memory: {:,}".format(
#            func.__name__,
#            mem_before, mem_after, mem_after - mem_before), "MB")
# 
#        return result
#    return wrapper
 
# instantiation of decorator function



***

# End of the notebook

In [ ]:
#import time

In [ ]:
#def display_usage(cpu_usage, mem_usage, bars=50):
#    cpu_percent = (cpu_usage / 100.0)
#    mem_percent = (mem_usage / 100.0)

#    cpu_bar     = '█' * int(cpu_percent * bars) + '-' * (bars - int(cpu_percent * bars))
#    mem_bar     = '█' * int(mem_percent * bars) + '-' * (bars - int(mem_percent * bars))
    
#    print(f'\rCPU Usage: |{cpu_bar}| {cpu_usage:.2f}%  ', end='')
#    print(f'MEM Usage: |{mem_bar}| {mem_usage:.2f}%  ', end='\r' )

In [ ]:
#while True:
#    display_usage(psutil.cpu_percent(), psutil.virtual_memory().percent, 30 )
#    time.sleep(0.5)